In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)



In [ ]:
## Creating folder architecture suitable for ImageAI tool
# for example: medical/train/CLASS_NAME/*.png
# medical: will be the parent directory
# CLASS_NAME will be 7-inch-bonney-forcep for example
# insde CLASS_NAME will be all related images and .xml files

In [ ]:
import os
import numpy as np
import shutil
import glob

In [ ]:
os.chdir("/content/drive/My Drive/multilabel_image_classification")

In [ ]:
! pip install imageai tensorflow-gpu==1.13.1

In [ ]:
! wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5
! pwd

In [ ]:
DATA_DIR = 'images'

for subdir in ['train', 'validation']:
  img_path = os.path.join(DATA_DIR, subdir, "images")
  anno_path = os.path.join(DATA_DIR, subdir, "annotations")

  for path in [img_path, anno_path]:
    for name in os.listdir(path):
      os.rename(os.path.join(path, name), os.path.join(path, name.replace(" ", "")))

  files1 = os.listdir(img_path)
  files2 = os.listdir(anno_path)

  files1 = [file.split('.')[0] for file in files1]
  files2 = [file.split('.')[0] for file in files2]

  for file in files1:
    if file not in files2:
      os.remove(os.path.join(img_path, file + '.png'))
  
  for file in files2:
    if file not in files1:
      os.remove(os.path.join(anno_path, file + '.xml'))

In [ ]:
# Training the model
from imageai.Detection.Custom import DetectionModelTrainer
#build the trainer as Yolo3
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
# define the parent folder
trainer.setDataDirectory(data_directory=DATA_DIR)

In [ ]:
PRETRAINED_MODEL_PATH = "pretrained-yolov3.h5"
trainer.setTrainConfig(object_names_array=unique_classes,
                       batch_size=8, # small batch size for small data
                       num_experiments=2, # num of epochs, incraese if the val_loss is decreasing well
                       train_from_pretrained_model=PRETRAINED_MODEL_PATH)
# begin training
trainer.trainModel()

In [ ]:
# evaluate the current model
MODEL_PATH = os.path.join(DATA_DIR, "models/detection_model-ex-027--loss-0004.255.h5")
CONFIG_PATH = os.path.join(DATA_DIR, "medical/json/detection_config.json"
from imageai.Detection.Custom import DetectionModelTrainer
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory=DATA_DIR)

metrics = trainer.evaluateModel(model_path=MODEL_PATH,
                                json_path=CONFIG_PATH,
                                iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)
print(metrics)

## Video Detection

In [ ]:
!cd medical
!pwd

In [ ]:
from imageai.Detection.Custom import CustomVideoObjectDetection
import os
import cv2

execution_path = os.getcwd()
camera = cv2.VideoCapture(-1)

detector = CustomVideoObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(os.path.join(execution_path , "medical/models/detection_model-ex-027--loss-0004.255.h5"))
detector.setJsonPath("medical/json/detection_config.json")
detector.loadModel()

video_path = detector.detectObjectsFromVideo(camera_input=camera,
    output_file_path=os.path.join(execution_path, "camera_detected_video")
    , frames_per_second=20, log_progress=True, minimum_percentage_probability=50)

print(video_path)

In [ ]:
from imageai.Detection.Custom import CustomVideoObjectDetection
import os

execution_path = os.getcwd()

video_detector = CustomVideoObjectDetection()
video_detector.setModelTypeAsYOLOv3()
video_detector.setModelPath("medical/models/detection_model-ex-027--loss-0004.255.h5")
video_detector.setJsonPath("medical/json/detection_config.json")
video_detector.loadModel()

video_detector.detectObjectsFromVideo(input_file_path=os.path.join(execution_path, "Ryder Needeholders - multiples.mp4"),
                                          output_file_path=os.path.join(execution_path, "detected1"),
                                          frames_per_second=20,  minimum_percentage_probability=40,
                                          detection_timeout=120,
                                          log_progress=True)

In [ ]:
from imageai.Detection.Custom import CustomVideoObjectDetection
import os

execution_path = os.getcwd()
def forFull(output_arrays, count_arrays, average_output_count):
    print("Array for the outputs of each frame ", output_arrays)
    print("Array for output count for unique objects in each frame : ", count_arrays)
    print("Output average count for unique objects in the entire video: ", average_output_count)
    print("------------END OF THE VIDEO --------------")

video_detector = CustomVideoObjectDetection()
video_detector.setModelTypeAsYOLOv3()
video_detector.setModelPath(os.path.join(execution_path, "medical/models/detection_model-ex-027--loss-0004.255.h5"))
video_detector.setJsonPath("medical/json/detection_config.json")
video_detector.loadModel()


video_detector.detectObjectsFromVideo(input_file_path=os.path.join(execution_path, "2 inch kelly retractor 2.mp4"), output_file_path=os.path.join(execution_path, "video_frame_analysis") ,  frames_per_second=30, video_complete_function=forFull,  minimum_percentage_probability=90)